In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgbm
import xgboost as xgb

from sklearn.ensemble import VotingClassifier

In [25]:
def plotRoc(predictions):
    predictions = predictions[['label','probability']]

    y = predictions['label']
    scores = predictions['probability']
    fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=1)
    
        # La guardo
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='Curva ROC')
    plt.plot([0, 1], [0, 1], color = 'navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Ratio Falsos Positivos')
    plt.ylabel('Ratio Verdaderos Positivos')
    plt.title('ROC')
    plt.legend(loc="lower right")
    plt.show()

In [26]:
def precissionRecall(y_test, predictions):
    df = pd.DataFrame({'label':y_test, 'prediction': predictions})
    verdaderos_positivos = len(df[(df['label'] == 1) & (df['prediction'] == 1)])
    falsos_positivos = len(df[(df['label'] == 0) & (df['prediction'] == 1)])
    falsos_negativos = len(df[(df['label'] == 1) & (df['prediction'] == 0)])
    precission = "\nPrecission: {}".format(verdaderos_positivos/(verdaderos_positivos+falsos_positivos))
    recall = "\nRecall: {}".format(verdaderos_positivos/(verdaderos_positivos+falsos_negativos))
    
    return str(precission+recall)

In [27]:
def resultados_modelo(model, X_test, y_test):
    predictions = model.predict(X_test)
    probabilities = model.predict_proba(X_test)
    preds = pd.DataFrame({'label': list(y_test), 'probability': list(probabilities)})
    preds['probability'] = preds['probability'].apply(lambda x: x[1])

    plotRoc(preds)
    info = ""
    info = info + ('Nº Variables: {}'.format(len(X_test.columns)))
    info = info + ("\nScore: {}".format(model.score(X_test, y_test)))

    from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, f1_score

    info = info + ('\nConfusion Matrix\n'+str(confusion_matrix(y_test, predictions)))

    fpr, tpr, thresholds = roc_curve(y_test, predictions)
    info = info + ('\nROC : {}'.format(roc_auc_score(y_test, preds['probability'] )))
    info = info + ('\nF1 : {}'.format(f1_score(y_test, predictions)))
    info = info + (precissionRecall(y_test, predictions))
#     info = info + ('\noob_decision_function_: {}'.format(model.oob_decision_function_))
    # info = info + ('\estimators_: {}'.format(model.estimators_))
    # info = info + ('\estimator_params: {}'.format(model.estimator_params))
    print(info)

In [28]:
def metricas(x_train, y_train,x_test, y_test, model, label_modelo = 'modelo'):
    
    # Train
    y_pred_train = model.predict(x_train)
    
    # Test
    y_pred_test = model.predict(x_test)
    
    resultados = pd.DataFrame({'Modelo': [label_modelo + '_train', label_modelo + '_test'],
                               'Precision' : [metrics.precision_score(y_train, y_pred_train), metrics.precision_score(y_test, y_pred_test)], 
                               'Recall':[metrics.recall_score(y_train, y_pred_train), metrics.recall_score(y_test, y_pred_test)], 
                               'F1': [metrics.f1_score(y_train, y_pred_train),metrics.f1_score(y_test, y_pred_test)],
                               'AUC': [roc_auc_score(y_train, y_pred_train), roc_auc_score(y_test, y_pred_test)]}, 
                              index = [0,1])
    return resultados

# metricas(x_test, y_test, rf, 'rf')

# Lectura del dataset y split train/test

In [29]:
path_lectura = r'C:\Users\admin_eci2019\notebooks\Proyecto_ECI\Data\05_con_variables_continuas_categorizadas'
path_guardado = r'C:\Users\admin_eci2019\notebooks\Proyecto_ECI\Data\05_con_variables_continuas_categorizadas'

In [30]:
df = pd.read_pickle(path_lectura + r'\df_train_hogar.pkl')
df = pd.get_dummies(df)

In [31]:
X = df.drop(['target'],axis=1).copy()
y = df['target'].copy()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 22)

In [39]:
pandas_profiling.ProfileReport(df)

C:\Anaconda\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,64
Number of observations,502725
Total Missing (%),0.0%
Total size in memory,68.1 MiB
Average record size in memory,142.0 B
Numeric,2
Categorical,0
Boolean,59
Date,0
Text (Unique),0
Rejected,3


In [45]:
# profile = df.profile_report(title = 'hola')
# profile.to_file(outputfile="Report_ECI-Hogar.html") 


# Modelo Regresión Logística

In [21]:
from sklearn.linear_model import LogisticRegression

model_logistic = LogisticRegression(class_weight='balanced',
                                    solver = 'sag', # For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones
                                    random_state=22,
                                    n_jobs=-1)

model_logistic.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=22, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
datos_metricas = metricas(X_train, y_train,X_test, y_test, model_logistic, 'logistica')
datos_metricas

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Modelo,Precision,Recall,F1,AUC
0,logistica_train,0.0,0.0,0.0,0.499996
1,logistica_test,0.0,0.0,0.0,0.500000


# Modelo BaggingClassifier con DecisionTree

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [ ]:
model_bagging = BaggingClassifier(DecisionTreeClassifier(criterion = 'gini',
                                                       max_depth = 17,
                                                       max_features = 'auto',
                                                       random_state=1))

model_baggin.fit(X_train, y_train)

In [ ]:
# Poner la optimización de los hiperparámetros
max_depth = [10, 13, 15, 20]
# min_weight_fraction_leaf = [0.003]
criterion = ['gini', 'entropy']
min_samples_split = [100, 150, 200]
min_samples_leaf = [15, 20, 25] 
max_leaf_nodes = [2**8, 2**12, 2**14, 2**16]
max_features = ['auto']
class_weight = [{1:4},{1:10},'balanced']

hyperF = dict(
              criterion = criterion, 
              max_depth = max_depth,  
              min_samples_split = min_samples_split, 
              min_samples_leaf = min_samples_leaf, 
              min_weight_fraction_leaf = min_weight_fraction_leaf,
              max_features = max_features,
#               max_leaf_nodes = max_leaf_nodes,
              class_weight = class_weight
)

model_dt = BaggingClassifier(DecisionTreeRegressor(criterion = 'gini',
                                 max_depth = 17,
                                 max_features = 'auto',
                                 random_state=1))

gridF = GridSearchCV(clf_over, hyperF, cv = 3, scoring = 'f1', verbose = 1, n_jobs = -1)
bestF = gridF.fit(X_train, y_train)

In [ ]:
dtm = bestF.best_estimator_.fit(X_train,y_train)

# Modelo RandomForest Classifier


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
model_rfc = RandomForestClassifier(n_estimators=200,
                                   criterion='gini',
                                   max_depth=17, 
                                   min_samples_split=100,
                                   min_samples_leaf=1, 
                                   min_weight_fraction_leaf=0.05,
                                   max_features='auto', n_jobs=None,
                                   random_state=1, verbose=0, 
                                   class_weight=[{1:2},'balanced'])

model_rfc.fit(X_train, y_train)

# Modelo XGBoost

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
import sklearn.ensemble

model_xgb = GradientBoostingClassifier(loss='deviance', # 'deviance', 'exponential'
                                       # learning_rate=0.1,
                                       n_estimators=200, 
                                       criterion='friedman_mse', # 'mse', 'friedman_mse', 'mae'
                                       min_samples_split=2,
                                       min_samples_leaf=1,
                                       min_weight_fraction_leaf=0.0,
                                       max_depth=3, 
                                       init=None,
                                       random_state=None,
                                       max_features=None,
                                       verbose=0, 
                                       max_leaf_nodes=None,
                                       warm_start=False,
                                       presort='auto',
                                       n_iter_no_change=None,
                                       tol=0.0001)

In [ ]:
best_xgb = model_xgb.fit(X_train, y_train)


# Modelo LightGBM

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMModel.html#lightgbm.LGBMModel

In [ ]:
import lightgbm as lgb

params = {}

model_lgb = lgb.LGBMModel(boosting_type='gbdt', # ‘gbdt’, ‘dart’, ‘goss’, ‘rf’.
                          num_leaves=31, # minimo de hojas del arbol
                          max_depth=20, 
                          n_estimators=200, 
                          objective=None,
                          class_weight=[{1:2},'balanced'],
                          colsample_bytree=1.0,
                          reg_alpha=0.0, 
                          reg_lambda=0.0, 
                          random_state=1,
                          n_jobs=-1, 
                          silent=False, 
                          importance_type='split' # 'split','gain'
                         )

In [ ]:
best_lgb = model_lgb.fit(X_train, y_train)

In [2]:
model_gbm()

# Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
# model1 = LogisticRegression(random_state=1)
# model2 = tree.DecisionTreeClassifier(random_state=1)
# model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
model.fit(X_train,y_train)
model.score(X_test,y_test)